In [ ]:
import numpy as np
import pandas as pd

def start():
    global name
    global z
    name=input(prompt="""
What is the file name?
""")
    file=f"""./{name}.xlsx"""
    types={"Sales": float}
    df=pd.read_excel(io = file,
                    header=0,
                    dtypes=types)
    for col in df:
        if df[col].isnull().sum().astype(int)>0:
            print("""
There are missing values in your data.
Action recommended before running data 
through transformation
""")
            print(df.isnull().sum())
        else:
            continue
    print(df.head(n=5))
    data=str.lower(input(prompt="""
Is the data monthly or quarterly?
"""))
    if "mon" in data:
        z=11
        mon_trans()
    elif "q" in data:
        z=3
        qrt_trans()

def mon_trans():

    file=f"""./{name}.xlsx"""
    types={"Sales": float}
    df=pd.read_excel(io = file,
                    header=0,
                    dtypes=types)

    df.index = df.index + 1
    #Creating moving average

    df["Moving_avg"]=0
    y=0
    x=z
    while x<len(df):
        df.iloc[x,-1]=round(df.iloc[y:x+1,-2].mean(),ndigits=2)
        y+=1
        x+=1

    #Creating seasonal factor

    df["Seasonal_factor"]=0

    x=z

    while x<len(df):
        df.iloc[x,-1]=round(df.iloc[x,-3]/df.iloc[x,-2],ndigits=2)
        x+=1

    #Creating a seperate column with just the months

    df["Month"]=df["Date"].dt.month


    #Find the average seasonal factor

    pivot=pd.pivot_table(df.iloc[z:,:],index=["Month"],values=["Seasonal_factor"])

    season=[]
    for i,j in pivot.iterrows():
        season.append(j["Seasonal_factor"])


    df["Seasonal_factor2"]=0

    x=0

    while x<len(df):
        df.iloc[x,-1]=season[(df.iloc[x,-2])-1]
        x+=1


    df["Deseasonal"]=0

    x=0

    while x<len(df):
        df.iloc[x,-1]=round(df.iloc[x,1]/df.iloc[x,-2],ndigits=2)
        x+=1
    #Filtering out the data not usable anymore   
    df=df[["Date","Sales","Deseasonal"]]


    #Creating Dummies

    df["Year"]=df["Date"].dt.year.astype(int)
    df["Month"]=df["Date"].dt.month.astype(int)

    Months=["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov"]
    y=0

    while y<len(Months):
        df[Months[y]]=0

        x=0
        while x<len(df):
            if df.iloc[x,4]==y+1:
                df.iloc[x,-1]=1
                x+=1
            else:
                x+=1
                continue

        y+=1
        continue

    #Creating the linear dataframe    
    lin=pd.DataFrame.copy(df)
    lin=lin[["Sales","Deseasonal","Year","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Date"]]

    #Transforming for quadratic dataframe
    quad=pd.DataFrame.copy(df)

    quad["Obs^2"]=0

    x=0

    while x<len(quad):
        quad.iloc[x,-1]=quad.index[x]**2
        x+=1

    quad["Year^2"]=0

    x=0

    while x<len(quad):
        quad.iloc[x,-1]=quad.iloc[x,3]**2
        x+=1
    quad=quad[["Obs^2","Sales","Deseasonal","Year","Year^2","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Date"]]

    #transforminig for exponential dataframe

    exp=pd.DataFrame.copy(df)

    exp["ln_Sales"]=0

    x=0

    while x<len(exp):
        exp.iloc[x,-1]=np.log(exp.iloc[x,1])
        x+=1


    exp["ln_Deseasonal"]=0

    x=0

    while x<len(exp):
        exp.iloc[x,-1]=np.log(exp.iloc[x,2])
        x+=1

    exp=exp[["ln_Sales","ln_Deseasonal","Year","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Date"]]

    #Storing data in excel

    writer = pd.ExcelWriter('monthly_regression_input.xlsx', engine='xlsxwriter')

    lin.to_excel(writer, sheet_name="Linear_Input")
    quad.to_excel(writer, sheet_name="Quadratic_Input")
    exp.to_excel(writer, sheet_name="Exponential_Input")

    writer.save()
    
    
def qrt_trans():
    file=f"""./{name}.xlsx"""
    types={"Sales": float}
    df=pd.read_excel(io = file,
                    header=0,
                    dtypes=types)
    df.index = df.index + 1
    try:
        pd.to_numeric(df["Quarter"])
        #print("to numeric")
    except:
        qrt_lst=[]
        df["Quarter"]=df["Quarter"].str.split("Q")
        #print("transform")
        x=0
        for row in df["Quarter"]:
            qrt_lst.append(row[-1])
            x+=1
        qrt_int=[int(i) for i in qrt_lst]
        df["Q"]=0
        x=0

        for row in df["Q"]:
            df.iloc[x,-1]=qrt_int[x]
            x+=1
        df["Quarter"]=df["Q"]
    df=df[["Year","Quarter","Sales"]]


    df["Moving_avg"]=0
    y=0
    x=z
    while x<len(df):
        df.iloc[x,-1]=round(df.iloc[y:x+1,-2].mean(),ndigits=2)
        y+=1
        x+=1

    #Creating seasonal factor

    df["Seasonal_factor"]=0

    x=z

    while x<len(df):
        df.iloc[x,-1]=round(df.iloc[x,-3]/df.iloc[x,-2],ndigits=2)
        x+=1



    #Find the average seasonal factor

    pivot=pd.pivot_table(df.iloc[z:,:],index=["Quarter"],values=["Seasonal_factor"])

    season=[]
    for i,j in pivot.iterrows():
        season.append(j["Seasonal_factor"])


    df["Seasonal_factor2"]=0


    x=0

    while x<len(df):
        df.iloc[x,-1]=season[(df.iloc[x,1])-1]
        x+=1


    df["Deseasonal"]=0

    x=0

    while x<len(df):
        df.iloc[x,-1]=round(df.iloc[x,2]/df.iloc[x,-2],ndigits=2)
        x+=1
    # 


    qrt=["Q1","Q2","Q3"]


    y=0

    while y<len(qrt):
        df[qrt[y]]=0

        x=0
        while x<len(df):
            if df.iloc[x,1]==y+1:
                df.iloc[x,-1]=1
                x+=1
            else:
                x+=1
                continue

        y+=1
        continue

    df=df[["Sales","Deseasonal","Year","Q1","Q2","Q3"]]
    
    quad=pd.DataFrame.copy(df)

    quad["Obs^2"]=0

    x=0

    while x<len(quad):
        quad.iloc[x,-1]=quad.index[x]**2
        x+=1

    quad["Year^2"]=0

    x=0

    while x<len(quad):
        quad.iloc[x,-1]=quad.iloc[x,2]**2
        x+=1
    quad=quad[["Obs^2","Sales","Deseasonal","Year","Year^2","Q1","Q2","Q3"]]

    #transforminig for exponential dataframe

    exp=pd.DataFrame.copy(df)

    exp["ln_Sales"]=0

    x=0

    while x<len(exp):
        exp.iloc[x,-1]=np.log(exp.iloc[x,0])
        x+=1


    exp["ln_Deseasonal"]=0

    x=0

    while x<len(exp):
        exp.iloc[x,-1]=np.log(exp.iloc[x,1])
        x+=1

    exp=exp[["ln_Sales","ln_Deseasonal","Year","Q1","Q2","Q3"]]

    #Storing data in excel
    
    
    writer = pd.ExcelWriter('quarterly_regression_input.xlsx', engine='xlsxwriter')

    df.to_excel(writer, sheet_name="Linear_Input")
    quad.to_excel(writer, sheet_name="Quadratic_Input")
    exp.to_excel(writer, sheet_name="Exponential_Input")

    writer.save()

start()